In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
os.chdir("/mount/studenten-temp1/users/godberja/SpokenDialogSystems/college_adviser") # TODO use own path towards the git repo

from soloist.examples.college_bot.soloist.server import *
from soloist.examples.college_bot.collegebot_server import *

In [2]:
import json
from nltk.translate.bleu_score import corpus_bleu
import csv

In [3]:
# load the test dialogues (note that they have been generated where all automatically generated dialogues were generated (see data folder))
with open("/mount/studenten-temp1/users/godberja/SpokenDialogSystems/college_adviser/evaluation/test_dialogues.json","r",encoding="utf8") as infile:
    automatic_test_data = json.load(infile)
# also load the test data containing the manual test dialogues
with open("/mount/studenten-temp1/users/godberja/SpokenDialogSystems/college_adviser/evaluation/manual_test_dialogues.json","r",encoding="utf8") as infile:
    manual_test_data = json.load(infile)

In [6]:
# note that this ipynb-file is re-run multiple times with the according model-paths
args.model_name_or_path = '/mount/studenten-temp1/users/zabereus/adviser/soloist_env/soloist/examples/college_bot/college_model'
# args.model_name_or_path = '/mount/studenten-temp1/users/zabereus/adviser/soloist_env/soloist/examples/college_bot/college_model_manual_only'
# args.model_name_or_path = '/mount/studenten-temp1/users/zabereus/adviser/soloist_env/soloist/examples/college_bot/soloist/gtg_pretrained'
main()

Some weights of the model checkpoint at /mount/studenten-temp1/users/zabereus/adviser/soloist_env/soloist/examples/college_bot/college_model were not used when initializing GPT2LMHeadModel: ['multiple_choice_head.summary.bias', 'multiple_choice_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
response_ex, beliefstate_ex = get_response(["i'm looking for a college in new haven"])
print(response_ex)
print(beliefstate_ex)
print()
response_ex, beliefstate_ex = get_response(["i want to study architecture"])
print(response_ex)
print(beliefstate_ex)
print()
response_ex, beliefstate_ex = get_response(["i'm looking for a college in new haven","which subject would you like to study there ?","i want to study architecture"])
print(response_ex)
print(beliefstate_ex)

sampled results [' state = new haven, options = 2system : alright. there is a lot of colleges left. what specifically are you looking for?']
response alright . there is a lot of colleges left . what specifically are you looking for ?, bs {'state': 'new haven', 'options': '2'}
1
sampled results ['system : [name] is a [control] top university in [city], [state] with [students] undergraduate students. find out mor']
response [name] is a [control] top university in [city] , [state] with [students] undergraduate students . find out mor, bs {}
yale university is a private top university in new haven , connecticut with 6500 undergraduate students . find out mor
{'state': 'new haven', 'options': '2'}

sampled results [' area = architecture, options = 49system : would you like to narrow your search?']
response would you like to narrow your search ?, bs {'area': 'architecture', 'options': '49'}
28
sampled results ['system : do you have any other requirements for the college?']
response do you ha

In [5]:
# make the predictions and save them to file
predictions = []
for dialogue in automatic_test_data:
    prediction = dict()
    prediction = copy.deepcopy(dialogue)
    reply, belief = get_response(prediction["history"])
    prediction["predicted_belief"] = belief
    prediction["predicted_reply"] = reply
    predictions += [prediction]
# with open("predictions.json","w",encoding="utf8") as outfile:
#     json.dump(predictions, outfile,indent=4)

sampled results [' event_name = virginia ; event_type = Musicsystem : slot_venue is the event.user : what other events do you have i']
response slot_venue is the event.user : what other events do you have i, bs {'event_name': 'virginia', 'event_type': 'Music'}
event_name is not a valid slot
event_type is not a valid slot
49
sampled results ['7th Street Y ; event_name = mariners vs angels ; number_of_tickets = 1 ; event_type = Music 7 day']
response 7th Street Y ; event_name = mariners vs angels ; number_of_tickets = 1 ; event_type = Music 7 day, bs {'event_name': 'mariners vs angels', 'number_of_tickets': '1', 'event_type': 'Music 7 day'}
sampled results [" category = Music, Sports, or anything else you can think of.user : I've found 1 event called slot_event_name at slot"]
response category = Music , Sports , or anything else you can think of.user : I 've found 1 event called slot_event_name at slot, bs {'category': 'Music'}
category is not a valid slot
49
sampled results ['7 ; city_o

IndexError: list index out of range

In [ ]:
# with open("predictions_automatic.json","r",encoding="utf8") as infile:
#     predictions = json.load(infile)

Belief-State Analysis

In [6]:
# check accuracy of belief state (prediction)
def check_bs(gold_bs:str,predicted_bs:str,want_print=False):
    # we want to check true positives (gold+pred), false positive (only pred) and false negatives (only gold) for predicting slots and/or values
    slots_TP, slots_FP, slots_FN = 0, 0, 0
    values_TP, values_FP, values_FN = 0, 0, 0
    sv_TP, sv_FP, sv_FN = 0, 0, 0

    # transform gold belief state into dictionary
    gold = dict()
    gold_bs = gold_bs.replace("belief : ","")
    gold_list = gold_bs.split(";")
    for slot_value in gold_list:
        if "=" in slot_value:
            splitted = slot_value.split("=")
            slot = splitted[0].strip()
            value = splitted[1].strip()
            if slot not in gold:
                gold[slot] = [value]
            else:
                gold[slot] += [value]
    if want_print:
        print("gold: ",gold)

    # transform predicted belief state into dictionary
    predicted = dict()
    predicted_bs = predicted_bs.replace("belief : ","")
    predicted_list = predicted_bs.split(";")
    for slot_value in predicted_list:
        if "=" in slot_value:
            splitted = slot_value.split("=")
            slot = splitted[0].strip()
            value = splitted[1].strip()
            if slot not in predicted:
                predicted[slot] = [value]
            else:
                predicted[slot] += [value]
    if want_print:
        print("predicted: ",predicted)

    # consider counts for slots -> don't care about how many values were recognized / if they are correct; only recognition of slots matter
    gold_slots = list(gold.keys())
    predicted_slots = list(predicted.keys())
    slots_FP = len([item for item in predicted_slots if item not in gold_slots])
    slots_FN = len([item for item in gold_slots if item not in predicted_slots])
    slots_TP = len([item for item in predicted_slots if item in gold_slots]) 
    if want_print:
        print("slots:\tFP: ",slots_FP,"\tFN: ",slots_FN,"\tTP: ",slots_TP)

    # consider counts for values -> don't care if the values are matched properly to the slots; only recognition of values matter
    gold_values = [val for vals in gold.values() for val in vals]
    predicted_values = [val for vals in predicted.values() for val in vals]
    values_FP = len([item for item in predicted_values if item not in gold_values])
    values_FN = len([item for item in gold_values if item not in predicted_values])
    values_TP = len([item for item in predicted_values if item in gold_values]) 
    if want_print:
        print("values:\tFP: ",values_FP,"\tFN: ",values_FN,"\tTP: ",values_TP)

    # consider counts for slot-value-tuples -> slot as well as value need to match!
    gold_tuples = [(s,v) for s,vals in gold.items() for v in vals]
    predicted_tuples = [(s,v) for s,vals in predicted.items() for v in vals]
    sv_FP = len([item for item in predicted_tuples if item not in gold_tuples])
    sv_FN = len([item for item in gold_tuples if item not in predicted_tuples])
    sv_TP = len([item for item in predicted_tuples if item in gold_tuples]) 
    if want_print:
        print("slot-value pairs:\tFP: ",sv_FP,"\tFN: ",sv_FN,"\tTP: ",sv_TP)

    return {"slots":(slots_FP,slots_FN,slots_TP),"values":(values_FP,values_FN,values_TP),"slots_values":(sv_FP,sv_FN,sv_TP)}

In [44]:
# run some examples
check_bs("belief : area = architecture ; state = california"," belief : state = california",want_print=True)
print()
check_bs("belief : area = architecture"," belief : state = california",want_print=True)
print()
check_bs("belief : state = california"," belief : area = architecture ;  state = california",want_print=True)
print()
check_bs("belief : area = architecture ; area = engineering"," belief : area = architecture",want_print=True)
print()
check_bs("belief : area = architecture ; area = engineering"," belief : area = architecture ; area = engineering",want_print=True)
print()
check_bs("belief : area = architecture ; state = california"," belief : state = california",want_print=True)

gold:  {'area': ['architecture'], 'state': ['california']}
predicted:  {'state': ['california']}
slots:	FP:  0 	FN:  1 	TP:  1
values:	FP:  0 	FN:  1 	TP:  1
slot-value pairs:	FP:  0 	FN:  1 	TP:  1

gold:  {'area': ['architecture']}
predicted:  {'state': ['california']}
slots:	FP:  1 	FN:  1 	TP:  0
values:	FP:  1 	FN:  1 	TP:  0
slot-value pairs:	FP:  1 	FN:  1 	TP:  0

gold:  {'state': ['california']}
predicted:  {'area': ['architecture'], 'state': ['california']}
slots:	FP:  1 	FN:  0 	TP:  1
values:	FP:  1 	FN:  0 	TP:  1
slot-value pairs:	FP:  1 	FN:  0 	TP:  1

gold:  {'area': ['architecture', 'engineering']}
predicted:  {'area': ['architecture']}
slots:	FP:  0 	FN:  0 	TP:  1
values:	FP:  0 	FN:  1 	TP:  1
slot-value pairs:	FP:  0 	FN:  1 	TP:  1

gold:  {'area': ['architecture', 'engineering']}
predicted:  {'area': ['architecture', 'engineering']}
slots:	FP:  0 	FN:  0 	TP:  1
values:	FP:  0 	FN:  0 	TP:  2
slot-value pairs:	FP:  0 	FN:  0 	TP:  2

gold:  {'area': ['architectu

{'slots': (0, 1, 1), 'values': (0, 1, 1), 'slots_values': (0, 1, 1)}

In [7]:
def get_prec_rec_f(fp,fn,tp,want_print=False):
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    f1 = 2 * ((prec * rec) / (prec + rec))
    if want_print:
        print("prec: ",prec,"\trec: ",rec,"\tf1: ",f1)

In [8]:
# iterate over test data, make soloist predictions and get precision, recall and f1-score
slots = {"FP":0,"FN":0,"TP":0}
values = {"FP":0,"FN":0,"TP":0}
slots_values = {"FP":0,"FN":0,"TP":0}
for test_dialogue in predictions:
    gold_bs = test_dialogue["belief"]
    gold_bs = str("belief : "+gold_bs)
    predicted_bs = test_dialogue["predicted_belief"]
    predicted_bs_temp = "belief : "
    for s,v in predicted_bs.items():
        if len(predicted_bs_temp) == 9:
            predicted_bs_temp += str(s + " = " + v)
        else:
            predicted_bs_temp += str(" ; " + s + " = " + v)
    predicted_bs = predicted_bs_temp
    counts = check_bs(gold_bs,predicted_bs)
    slots["FP"] += counts["slots"][0]
    slots["FN"] += counts["slots"][1]
    slots["TP"] += counts["slots"][2]
    values["FP"] += counts["values"][0]
    values["FN"] += counts["values"][1]
    values["TP"] += counts["values"][2]
    slots_values["FP"] += counts["slots_values"][0]
    slots_values["FN"] += counts["slots_values"][1]
    slots_values["TP"] += counts["slots_values"][2]
print("slots: ")
print("\t",slots)
print("\t",get_prec_rec_f(slots["FP"],slots["FN"],slots["TP"],True))
print("values: ")
print("\t",values)
print("\t",get_prec_rec_f(values["FP"],values["FN"],values["TP"],True))
print("slot values: ")
print("\t",slots_values)
print("\t",get_prec_rec_f(slots_values["FP"],slots_values["FN"],slots_values["TP"],True))


slots: 
	 {'FP': 493, 'FN': 124, 'TP': 114}
prec:  0.18780889621087316 	rec:  0.4789915966386555 	f1:  0.2698224852071006
	 None
values: 
	 {'FP': 461, 'FN': 102, 'TP': 146}
prec:  0.24052718286655683 	rec:  0.5887096774193549 	f1:  0.3415204678362573
	 None
slot values: 
	 {'FP': 511, 'FN': 152, 'TP': 96}
prec:  0.15815485996705106 	rec:  0.3870967741935484 	f1:  0.22456140350877193
	 None


Analysis of system output

In [32]:
# example
gold_out_ex = "system : there's many. is there anything else you want from the college?"
gold_out_ex = gold_out_ex.replace("system : ","")
predicted_out_ex = "alright . there is a lot of colleges left . what specifically are you looking for ?"
bleu_ex = corpus_bleu([gold_out_ex.split()],[predicted_out_ex.split()])
print(bleu_ex)

1.1808001696991507e-231


/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram o

In [9]:
bleu_scores = [] # collegt bleu scores for all test-dialogues
golds_out = []
preds_out = []
for test_dialogue in predictions:
    gold_out = test_dialogue["reply"]
    predicted_out = test_dialogue["predicted_reply"]
    if "system" in gold_out:
        gold_out = gold_out.replace("system : ","")
        if len("system") > 1: # we actually have some gold-system-reply
            golds_out += [gold_out.split()]
            preds_out += [predicted_out.split()]
bleu_score = corpus_bleu(golds_out,preds_out)
print(bleu_score)

1.0598704614402735e-231


/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the B

The BLEU scores will be too much effort (as we would need (multiple) reference outputs). Also the values would be way to small. Thus for the evaluation of the actual system output we will only rely on doing our own manual evaluations (and averaging the values).

In [9]:
with open("evaluation/responses_college_model_manualOnly.csv","w",newline="") as outfile:
    writer = csv.writer(outfile)
    field = ["type","question","response"]
    writer.writerow(field)
    # only evaluate the base cases
    for dialogue in manual_test_data:
        dialogue_id = list(dialogue.keys())[0]
        dialogue_content = dialogue[dialogue_id]
        if "base" in dialogue_id:
            # print(dialogue_id)
            # print(dialogue_content)
            response,belief = get_response(dialogue_content)
            # print("response ",response)
            # print("belief ",belief)
            writer.writerow([dialogue_id,dialogue_content,response])

sampled results [' state = michigan ; area = business; name = cwru; price_range = pricey ; state = new york ; name ']
response state = michigan ; area = business ; name = cwru ; price_range = pricey ; state = new york ; name, bs {'state': 'new york', 'area': 'business', 'name': 'cwru', 'price_range': 'pricey'}
0


sampled results ["system : i couldn't find any colleges matching your query. would you like to try again?"]
response i could n't find any colleges matching your query . would you like to try again ?, bs {}
sampled results [' state = michigan ; area = business; name = university of michigan-ann arbor, options = 1system : p [nam']
response p [nam, bs {'state': 'michigan', 'area': 'business', 'name': 'university of michigan-ann arbor', 'options': '1'}
1
sampled results ["system : i couldn't find any colleges matching your query. would you like to try again?"]
response i could n't find any colleges matching your query . would you like to try again ?, bs {}
sampled results [' city = new york, options = 1system : s [name] is in [city], [state]']
response s [name] is in [city] , [state], bs {'city': 'new york', 'options': '1'}
7
sampled results ['system : i found one college that might interest you: [name] in [city]. do you want to know more about it?']
response i found one college that might

In [10]:
# evaluate base and edge cases
with open("evaluation/responses_edgecases_college_model.csv","w",newline="") as outfile:
    writer = csv.writer(outfile)
    field = ["type","question","response"]
    writer.writerow(field)
    # only evaluate the base cases
    for dialogue in manual_test_data:
        dialogue_id = list(dialogue.keys())[0]
        dialogue_content = dialogue[dialogue_id]
        # print(dialogue_id)
        # print(dialogue_content)
        response,belief = get_response(dialogue_content)
        # print("response ",response)
        # print("belief ",belief)
        writer.writerow([dialogue_id,dialogue_content,response])
        

sampled results [' state = michigan ; area = business; name = university of michigan-ann arbor, options = 1system : a [nam']
response a [nam, bs {'state': 'michigan', 'area': 'business', 'name': 'university of michigan-ann arbor', 'options': '1'}
1
sampled results ['system : [name] is a college in [city] and thus belongs to [region].']
response [name] is a college in [city] and thus belongs to [region] ., bs {}
sampled results [' state = michigan ; area = philosophy ; name = university of michigan-ann arbor, options = 1system : p [nam']
response p [nam, bs {'state': 'michigan', 'area': 'philosophy', 'name': 'university of michigan-ann arbor', 'options': '1'}
1
sampled results ['system : [name] is a [control] top university in [city], [state] with [students] undergraduate students. find out mor']
response [name] is a [control] top university in [city] , [state] with [students] undergraduate students . find out mor, bs {}
sampled results [' city = new york ; name = university of michigan

In [8]:
# only evaluate the base cases
for dialogue in manual_test_data:
    dialogue_id = list(dialogue.keys())[0]
    dialogue_content = dialogue[dialogue_id]
    if "base" in dialogue_id:
        # print(dialogue_id)
        print("-----------------------------")
        print(dialogue_content)
        response,belief = get_response(dialogue_content)
        print("response ",response)
        print("belief ",belief)
            # writer.writerow([dialogue_id,dialogue_content,response])

-----------------------------
Can you give me a university in California?
sampled results [' city = los angeles, name = university of southern california, options = 1system : a [name] is located in [city]']
response a [name] is located in [city], bs {'city': 'los angeles', 'name': 'university of southern california', 'options': '1'}
1
sampled results ['system : [name] is a [control] top university in [city], [state] with [students] undergraduate students. find out mor']
response [name] is a [control] top university in [city] , [state] with [students] undergraduate students . find out mor, bs {}
response  university of southern california is a private top university in los angeles , california with 20600 undergraduate students . find out mor
belief  {'city': 'los angeles', 'name': 'university of southern california', 'options': '1'}
-----------------------------
Can you give me a university in Pittsburgh?
sampled results [' state = michigan ; area = business; name = university of michig